In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:
import pandas as pd

eval_df = pd.read_csv('../data/eval_qas_with_node_ids.csv', index_col=0)
eval_df.head(5)

,question,answer,qa_type,ground_truth,node_ids
0,What distinguishes the Boat-billed Heron from ...,The Boat-billed Heron is overall much whiter i...,single_fact,"{'bird_name': 'Boat-billed Heron', 'title': 'I...",['4:a554891f-4e4e-45d4-beca-f7796c51940f:197847']
1,How can the Plain Prinia be distinguished from...,"By its smaller size, less bulky appearance, na...",single_fact,"{'bird_name': 'Plain Prinia', 'title': 'Identi...",['4:a554891f-4e4e-45d4-beca-f7796c51940f:457779']
2,What is a distinguishing feature of Monteiro's...,It has dark grey eyes (not yellow) and a more ...,single_fact,"{'bird_name': ""Monteiro's Bushshrike"", 'title'...",['4:a554891f-4e4e-45d4-beca-f7796c51940f:416657']
3,What are the key identification features of th...,"Large-headed, dark-olive flatbill with white e...",single_fact,"{'bird_name': 'Western Olivaceous Flatbill', '...",['4:a554891f-4e4e-45d4-beca-f7796c51940f:363358']
4,What are some distinguishing features of the E...,The European Honey-buzzard differs from true b...,single_fact,"{'bird_name': 'European Honey-buzzard', 'title...",['4:a554891f-4e4e-45d4-beca-f7796c51940f:209969']


In [4]:
from langsmith import Client
from dotenv import load_dotenv

load_dotenv()

client = Client()

dataset_name = 'BWE Evaluation Dataset'

if not client.has_dataset(dataset_name=dataset_name):
	eval_dataset = client.upload_dataframe(
		df=eval_df,
		name=dataset_name,
		input_keys=['question'],
		output_keys=['answer', 'qa_type', 'ground_truth', 'node_ids'],
		description='Bird Watchers Emporium evaluation dataset'
	)
else:
    eval_dataset = client.read_dataset(dataset_name=dataset_name)
	
eval_dataset

Dataset(name='BWE Evaluation Dataset', description='Bird Watchers Emporium evaluation dataset', data_type=<DataType.kv: 'kv'>, id=UUID('0f31cb48-e946-46bd-8dd2-cb740bcf5320'), created_at=datetime.datetime(2025, 11, 23, 15, 55, 48, 95154, tzinfo=datetime.timezone.utc), modified_at=datetime.datetime(2025, 11, 23, 15, 55, 48, 95154, tzinfo=datetime.timezone.utc), example_count=6336, session_count=0, last_session_start_time=None, inputs_schema=None, outputs_schema=None, transformations=None)

In [9]:
from collections import defaultdict

examples = list(client.list_examples(dataset_id=eval_dataset.id))

by_qa_type: dict[str, list[str]] = defaultdict(list)

for ex in examples:
	qa_type = ex.outputs['qa_type']
	by_qa_type[str(qa_type)].append(ex.id)

for qa_type, ex_ids in by_qa_type.items():
	client.update_dataset_splits(
		dataset_id=eval_dataset.id,
		split_name=qa_type,
		example_ids=ex_ids,
		remove=False,
	)
	print('Updated split: ', qa_type)

Updated split:  vsc_neighborhood
Updated split:  2_hop
Updated split:  3_hop
Updated split:  4_hop
Updated split:  single_fact


In [ ]:
### AI-generated code that syncs dataframe with dataset


from typing import Sequence, List, Dict, Any
from langsmith import Client
import pandas as pd

def sync_dataset_with_dataframe(
    client: Client,
    dataset_name: str,
    df: pd.DataFrame,
    input_keys: Sequence[str],
    output_keys: Sequence[str],
    key_cols: Sequence[str] = ("question",),
) -> None:
    """
    Sync a LangSmith dataset to match a pandas DataFrame.

    - Adds new examples for new rows.
    - Deletes examples for rows that disappeared.
    - Updates examples for rows that changed (including new columns).

    key_cols: columns that uniquely identify a row (e.g. ["question"])
    """
    # Ensure dataset exists
    dataset = client.read_dataset(dataset_name=dataset_name)

    # --- Build key -> row mapping from DataFrame ---
    df = df.copy()

    # Create a stable string key per row
    df["__key__"] = df[list(key_cols)].astype(str).agg("||".join, axis=1)
    df_records: Dict[str, Dict[str, Any]] = {
        key: row for key, row in zip(df["__key__"], df.to_dict(orient="records"))
    }

    df_keys = set(df_records.keys())

    # --- Build key -> example mapping from LangSmith ---
    examples = list(client.list_examples(dataset_id=dataset.id))
    existing_by_key: Dict[str, Any] = {}

    for ex in examples:
        # Derive key from example inputs using the same key_cols
        key_vals = [ex.inputs.get(col) for col in key_cols]
        key = "||".join(str(v) for v in key_vals)
        existing_by_key[key] = ex

    existing_keys = set(existing_by_key.keys())

    # --- Diff ---
    keys_to_add = df_keys - existing_keys
    keys_to_delete = existing_keys - df_keys
    keys_to_update = df_keys & existing_keys

    # --- Add new examples ---
    examples_to_create: List[Dict[str, Any]] = []
    for key in keys_to_add:
        row = df_records[key]
        inputs = {k: row.get(k) for k in input_keys if k in row}
        outputs = {k: row.get(k) for k in output_keys if k in row}
        examples_to_create.append({"inputs": inputs, "outputs": outputs})

    if examples_to_create:
        client.create_examples(dataset_id=dataset.id, examples=examples_to_create)

    # --- Delete removed examples ---
    if keys_to_delete:
        delete_ids = [existing_by_key[k].id for k in keys_to_delete]
        client.delete_examples(example_ids=delete_ids)

    # --- Update existing examples (incl. new columns/changed values) ---
    if keys_to_update:
        update_ids: List[str] = []
        update_inputs: List[Dict[str, Any]] = []
        update_outputs: List[Dict[str, Any]] = []

        for key in keys_to_update:
            row = df_records[key]
            ex = existing_by_key[key]

            new_inputs = {k: row.get(k) for k in input_keys if k in row}
            new_outputs = {k: row.get(k) for k in output_keys if k in row}

            # Optional: you could diff ex.inputs/outputs vs new_* and skip identical ones.
            update_ids.append(ex.id)
            update_inputs.append(new_inputs)
            update_outputs.append(new_outputs)

        client.update_examples(
            example_ids=update_ids,
            inputs=update_inputs,
            outputs=update_outputs,
        )
